In [1]:
# !pip install emoji

In [2]:
import emoji
import numpy as np
import pandas as pd

In [3]:
mapping = pd.read_csv("C:/Users/Aheesha/Desktop/Cognorise/Task 2 - Emoji Prediction/Mapping.csv")
output = pd.read_csv("C:/Users/Aheesha/Desktop/Cognorise/Task 2 - Emoji Prediction/OutputFormat.csv")
train = pd.read_csv("C:/Users/Aheesha/Desktop/Cognorise/Task 2 - Emoji Prediction/Train.csv")
test = pd.read_csv("C:/Users/Aheesha/Desktop/Cognorise/Task 2 - Emoji Prediction/Test.csv")

In [4]:
mapping

,number,emoticons,number,Unnamed: 3
0,0,😜,0,NaN
1,1,📸,1,NaN
2,2,😍,2,NaN
3,3,😂,3,NaN
4,4,😉,4,NaN
5,5,🎄,5,NaN
6,6,📷,6,NaN
7,7,🔥,7,NaN
8,8,😘,8,NaN
9,9,❤,9,NaN


In [5]:
mapping = mapping.set_index('number')

In [6]:
mapping.head()

,emoticons,number,Unnamed: 3
number,,,
0,😜,0,NaN
1,📸,1,NaN
2,😍,2,NaN
3,😂,3,NaN
4,😉,4,NaN


In [7]:
mapping_dict = mapping.to_dict()
mapping_dict = mapping_dict['emoticons']
mapping_dict

{0: '😜',
 1: '📸',
 2: '😍',
 3: '😂',
 4: '😉',
 5: '🎄',
 6: '📷',
 7: '🔥',
 8: '😘',
 9: '❤',
 10: '😁',
 11: '🇺🇸',
 12: '☀',
 13: '✨',
 14: '💙',
 15: '💕',
 16: '😎',
 17: '😊',
 18: '💜',
 19: '💯'}

In [8]:
train.head()

,Unnamed: 0,TEXT,Unnamed: 2
0,NaN,Label,NaN
1,0.0,Vacation wasted ! #vacation2017 #photobomb #ti...,0.0
2,1.0,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1.0
3,2.0,Been friends since 7th grade. Look at us now w...,2.0
4,3.0,This is what it looks like when someone loves ...,3.0


In [9]:
train.columns

Index(['Unnamed: 0', 'TEXT', 'Unnamed: 2'], dtype='object')

In [10]:
train.tail()

,Unnamed: 0,TEXT,Unnamed: 2
69996,69995.0,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3.0
69997,69996.0,"I SEA you, Seattle @ Ballard Seafood Festival\n",16.0
69998,69997.0,If one of my daughters is wearing this and ask...,2.0
69999,69998.0,Guess who whoop people on THEIR homecoming?! #...,3.0
70000,69999.0,We Love you Robbie @ Heritage Memorial Cemeter...,14.0


In [11]:
# changing the labels to emoji
# train = train.replace({"Label": mapping_dict})

In [12]:
train.head()

,Unnamed: 0,TEXT,Unnamed: 2
0,NaN,Label,NaN
1,0.0,Vacation wasted ! #vacation2017 #photobomb #ti...,0.0
2,1.0,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1.0
3,2.0,Been friends since 7th grade. Look at us now w...,2.0
4,3.0,This is what it looks like when someone loves ...,3.0


In [13]:
emoji_mapping = {
    '0': ':beating_heart:',
    '1': ':baseball:',
    '2': ':beaming_face_with_smiling_eyes:',
    '3': ':angry_face:',
    '4': ':face_savoring_food:'
}

In [14]:
for key, value in emoji_mapping.items():
    emoji_mapping[key] = emoji.emojize(value)
    
print(emoji_mapping)

{'0': '💓', '1': '⚾', '2': '😁', '3': '😠', '4': '😋'}


In [15]:
def intialize_emb_matrix(file):
    embedding_matrix = {}
    for line in file:
        values = line.split()
        word = values[0]
        embedding = np.array(values[1:], dtype='float64')
        embedding_matrix[word] = embedding

    return embedding_matrix

In [16]:
def get_emb_data(data, max_len):
#     max_len = 168
    embedding_data = np.zeros((len(data), max_len, 50))  # from glove6B50d
    
    for idx in range(data.shape[0]):
        words_in_sentence = data[idx].split()
        
        for i in range(len(words_in_sentence)):
            if embedding_matrix.get(words_in_sentence[i].lower()) is not None:
                embedding_data[idx][i] = embedding_matrix[words_in_sentence[i].lower()]
                
    return embedding_data

In [17]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout

In [18]:
model = Sequential()

In [19]:
lm = len(mapping_dict)
lm

20

In [20]:
model.add(LSTM(units = 256, return_sequences=True, input_shape = (168,50)))
model.add(Dropout(0.3))
model.add(LSTM(units=128))
model.add(Dropout(0.3))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=20, activation='softmax'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 168, 256)          314368    
                                                                 
 dropout (Dropout)           (None, 168, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2

In [22]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [23]:
import pandas as pd
train_data = pd.read_csv("C:/Users/Aheesha/Desktop/Cognorise/Task 2 - Emoji Prediction/Train.csv", header=None)
test_data = pd.read_csv("C:/Users/Aheesha/Desktop/Cognorise/Task 2 - Emoji Prediction/Test.csv", header=None)

In [24]:
train_data.head()

,0,1,2
0,NaN,TEXT,NaN
1,NaN,Label,NaN
2,0.0,Vacation wasted ! #vacation2017 #photobomb #ti...,0.0
3,1.0,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1.0
4,2.0,Been friends since 7th grade. Look at us now w...,2.0


In [25]:
test_data.head()

,0,1,2
0,NaN,id,TEXT
1,0.0,0,Thought this was cool...#Repost (get_repost)・・...
2,1.0,1,Happy 4th! Corte madera parade. #everytownusa ...
3,2.0,2,Luv. Or at least something close to it. @ Unio...
4,3.0,3,There's a slice of pie under that whipped crea...


In [26]:
train_data.tail()

,0,1,2
69997,69995.0,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3.0
69998,69996.0,"I SEA you, Seattle @ Ballard Seafood Festival\n",16.0
69999,69997.0,If one of my daughters is wearing this and ask...,2.0
70000,69998.0,Guess who whoop people on THEIR homecoming?! #...,3.0
70001,69999.0,We Love you Robbie @ Heritage Memorial Cemeter...,14.0


In [27]:
emoji.emojize(':angry_face:')

'😠'

In [28]:
X_train = train_data[0].values
y_train = train_data[1].values

In [29]:
X_train

array([   nan,    nan,     0., ..., 69997., 69998., 69999.])

In [30]:
model_tte = Sequential()
model_tte.add(LSTM(units=64, input_shape=(10, 50), return_sequences=True))
model_tte.add(Dropout(0.3))
model_tte.add(LSTM(units=32))
model_tte.add(Dropout(0.2))
model_tte.add(Dense(units=10, activation='relu'))
model_tte.add(Dense(units=5, activation='softmax'))

In [31]:
model_tte.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 64)            29440     
                                                                 
 dropout_2 (Dropout)         (None, 10, 64)            0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 10)                330       
                                                                 
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
Total params: 42,241
Trainable params: 42,241
Non-trai

In [32]:
model_tte.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [33]:
model_tte2 = Sequential()
model_tte2.add(LSTM(units=128, input_shape=(10, 50), return_sequences=True))
model_tte2.add(Dropout(0.3))
model_tte2.add(LSTM(units=64))
model_tte2.add(Dropout(0.2))
model_tte2.add(Dense(units=32, activation='relu'))
model_tte2.add(Dense(units=20, activation='relu'))
model_tte2.add(Dense(units=5, activation='relu'))

In [34]:
model_tte2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 10, 128)           91648     
                                                                 
 dropout_4 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 20)                660       
                                                                 
 dense_9 (Dense)             (None, 5)                

In [35]:
model_tte2.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [36]:
model_tte3 = Sequential()
model_tte3.add(LSTM(units=128, input_shape=(10, 50), return_sequences=True))
model_tte3.add(Dropout(0.3))
model_tte3.add(LSTM(units=64))
model_tte3.add(Dropout(0.2))
model_tte3.add(Dense(units=32, activation='relu'))
model_tte3.add(Dense(units=20, activation='relu'))
model_tte3.add(Dense(units=5, activation='softmax'))

In [37]:
model_tte3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 10, 128)           91648     
                                                                 
 dropout_6 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 20)                660       
                                                                 
 dense_12 (Dense)            (None, 5)                

In [38]:
model_tte3.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])